In [48]:
import pandas as pd
import numpy as np
from ast import literal_eval
from sklearn.model_selection import train_test_split
import json

In [ ]:
df

In [2]:

df = pd.read_csv("../datasets/inversion_language_confusion/line_level/multi/train_data_clics3.csv")

In [49]:
with open("../datasets/inversion_language_confusion/line_level/multi/languages.json") as f:
    languages = json.load(f)

In [50]:
len(languages)

17

In [51]:
languages

['deu_Latn',
 'jpn_Jpan',
 'cmn_Hani',
 'eng_Latn',
 'tur_Latn',
 'urd_Arab',
 'arb_Arab',
 'hun_Latn',
 'fin_Latn',
 'pan_Guru',
 'guj_Gujr',
 'heb_Hebr',
 'kaz_Cyrl',
 'mon_Cyrl',
 'hin_Deva',
 'kor_Hang',
 'other']

In [5]:
len(df)

1789

In [6]:
df["clics3_pmi_norm"] = df["clics3_pmi_norm"].replace({-1:np.nan})

In [9]:
df = df.dropna(subset="clics3_pmi_norm")

In [29]:
df.head(2)

,model,training,eval_lang,step,pred_langs,eval_lang_encoded,script,family,script_lr,training_script_lr,...,pan_Guru,tur_Latn,urd_Arab,step_Base,step_Labels,step_Step1,step_Step50+sbeam8,training_iso3,eval_lang_iso3,clics3_pmi_norm
0,me5_cmn_jpn,"['cmn_Hani', 'jpn_Jpan']",deu_Latn,Step1,"{'deu_Latn': 0.87, 'jpn_Jpan': 0.07}",3,0,0,1,1,...,0,0,0,0.0,0.0,1.0,0.0,"['cmn', 'jpn']",deu,0.51216
1,me5_cmn_jpn,"['cmn_Hani', 'jpn_Jpan']",deu_Latn,Step50+sbeam8,"{'deu_Latn': 0.82, 'cmn_Hani': 0.07}",3,0,0,1,1,...,0,0,0,0.0,0.0,0.0,1.0,"['cmn', 'jpn']",deu,0.51216


In [18]:
df['pred_langs'] = df['pred_langs'].apply(literal_eval)

/var/folders/bj/qp6k2wl11h5gpn8j4hg0tgf40000gn/T/ipykernel_12307/3815735755.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pred_langs'] = df['pred_langs'].apply(literal_eval)


In [56]:
len(df), len(df.columns)

(856, 29)

In [57]:
len(df_cleaned), len(df_cleaned.columns)

(856, 28)

In [58]:
df = df.loc[:, (df != 0).any(axis=0)]

In [59]:
probs_df = pd.DataFrame(df['pred_langs'].tolist(), index=df.index).fillna(0)

In [60]:
probs_df["other"] = 1 - probs_df.sum(axis=1)

In [61]:
probs_df.head(2)

,deu_Latn,jpn_Jpan,cmn_Hani,eng_Latn,tur_Latn,urd_Arab,arb_Arab,hun_Latn,fin_Latn,heb_Hebr,kaz_Cyrl,mon_Cyrl,guj_Gujr,hin_Deva,kor_Hang,pan_Guru,other
0,0.87,0.07,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.06
1,0.82,0.00,0.07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.11


In [62]:
df.columns

Index(['model', 'training', 'eval_lang', 'step', 'pred_langs',
       'eval_lang_encoded', 'script', 'family', 'script_lr',
       'training_script_lr', 'arb_Arab', 'cmn_Hani', 'deu_Latn', 'guj_Gujr',
       'heb_Hebr', 'hin_Deva', 'jpn_Jpan', 'kaz_Cyrl', 'mon_Cyrl', 'pan_Guru',
       'tur_Latn', 'urd_Arab', 'step_Base', 'step_Step1', 'step_Step50+sbeam8',
       'training_iso3', 'eval_lang_iso3', 'clics3_pmi_norm'],
      dtype='object')

In [63]:
X = df.drop(columns=['model', 'eval_lang', 'step', 'pred_langs', "training", "training_iso3", 'eval_lang_iso3'])
y = probs_df

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:

from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error



In [66]:
regr = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)
mse = mean_squared_error(y_test, y_pred, multioutput="raw_values")
mae = mean_absolute_error(y_test, y_pred, multioutput="raw_values")

In [72]:
y_train

,deu_Latn,jpn_Jpan,cmn_Hani,eng_Latn,tur_Latn,urd_Arab,arb_Arab,hun_Latn,fin_Latn,heb_Hebr,kaz_Cyrl,mon_Cyrl,guj_Gujr,hin_Deva,kor_Hang,pan_Guru,other
7,0.00,0.0,0.94,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.06
381,0.00,0.0,0.00,0.06,0.37,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.50,0.00,0.0,0.0,0.07
324,0.00,0.0,0.00,0.00,0.08,0.0,0.00,0.00,0.0,0.00,0.85,0.0,0.00,0.00,0.0,0.0,0.07
605,0.66,0.0,0.00,0.00,0.25,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.09
243,0.00,0.0,0.09,0.29,0.00,0.0,0.14,0.00,0.0,0.08,0.00,0.0,0.00,0.00,0.0,0.0,0.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,0.00,0.0,0.00,0.00,0.84,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.16
251,0.12,0.0,0.00,0.08,0.09,0.0,0.12,0.14,0.0,0.07,0.00,0.0,0.00,0.00,0.0,0.0,0.38
507,0.00,0.0,0.00,0.72,0.08,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.08,0.0,0.0,0.12
854,0.00,0.0,0.00,0.17,0.00,0.0,0.00,0.00,0.0,0.00,0.17,0.0,0.48,0.00,0.0,0.0,0.18


In [73]:
features = X_train.columns
targets = y_train.columns

In [74]:
# Get feature names
feature_names = X_train.columns

# Create a dictionary to hold the mapping of target to feature importances
target_feature_importances = {}

# Accessing feature importances for each target
for i, estimator in enumerate(regr.estimators_):
    target_name = targets[i]  # Assign a target name (e.g., target1, target2)
    # Map feature names to their importances for this target
    feature_importance_map = dict(zip(feature_names, estimator.feature_importances_))
    # Add to the dictionary
    target_feature_importances[target_name] = feature_importance_map

# Display the mapped feature importances for each target
for target, importances in target_feature_importances.items():
    print(f"Feature importances for {target}:")
    for feature, importance in importances.items():
        print(f"{feature}: {importance:.4f}")
    print("\n")

Feature importances for deu_Latn:
eval_lang_encoded: 0.6185
script: 0.0191
family: 0.0033
script_lr: 0.0015
training_script_lr: 0.0005
arb_Arab: 0.0049
cmn_Hani: 0.0023
deu_Latn: 0.1287
guj_Gujr: 0.0021
heb_Hebr: 0.0011
hin_Deva: 0.0016
jpn_Jpan: 0.0010
kaz_Cyrl: 0.0005
mon_Cyrl: 0.0022
pan_Guru: 0.0008
tur_Latn: 0.1447
urd_Arab: 0.0008
step_Base: 0.0027
step_Step1: 0.0022
step_Step50+sbeam8: 0.0018
clics3_pmi_norm: 0.0597


Feature importances for jpn_Jpan:
eval_lang_encoded: 0.0718
script: 0.0252
family: 0.0543
script_lr: 0.0067
training_script_lr: 0.0000
arb_Arab: 0.0006
cmn_Hani: 0.0893
deu_Latn: 0.0019
guj_Gujr: 0.0000
heb_Hebr: 0.0001
hin_Deva: 0.0000
jpn_Jpan: 0.6031
kaz_Cyrl: 0.0000
mon_Cyrl: 0.0098
pan_Guru: 0.0000
tur_Latn: 0.0046
urd_Arab: 0.0000
step_Base: 0.0231
step_Step1: 0.0036
step_Step50+sbeam8: 0.0115
clics3_pmi_norm: 0.0944


Feature importances for cmn_Hani:
eval_lang_encoded: 0.0953
script: 0.0022
family: 0.0338
script_lr: 0.0119
training_script_lr: 0.0015
arb_Ara

In [76]:
df_results = pd.DataFrame(target_feature_importances)

In [77]:
df_results

,deu_Latn,jpn_Jpan,cmn_Hani,eng_Latn,tur_Latn,urd_Arab,arb_Arab,hun_Latn,fin_Latn,heb_Hebr,kaz_Cyrl,mon_Cyrl,guj_Gujr,hin_Deva,kor_Hang,pan_Guru,other
eval_lang_encoded,0.618523,0.071789,0.095284,0.359905,0.156129,0.271344,0.114213,0.251415,0.332193,0.048678,0.291994,0.209356,0.284301,0.240226,0.219489,0.285961,0.138147
script,0.019099,0.025221,0.002222,0.044446,0.030748,0.008742,0.105908,0.089652,0.013632,0.018613,0.019783,0.032989,0.008095,0.010806,0.000000,0.007234,0.038064
family,0.003299,0.054339,0.033821,0.012546,0.043377,0.119674,0.057172,0.000203,0.002087,0.001819,0.055283,0.042390,0.038073,0.023062,0.094826,0.056382,0.032384
script_lr,0.001512,0.006676,0.011871,0.040417,0.008258,0.000387,0.059662,0.000000,0.004245,0.013944,0.007350,0.017431,0.007734,0.012278,0.009840,0.010543,0.018056
training_script_lr,0.000525,0.000000,0.001480,0.012990,0.004470,0.108154,0.066728,0.001836,0.006590,0.000000,0.002833,0.000084,0.000000,0.006575,0.000000,0.000000,0.006529
arb_Arab,0.004906,0.000551,0.004245,0.012939,0.003938,0.009652,0.194592,0.002020,0.062513,0.347524,0.000514,0.009069,0.000000,0.000000,0.000000,0.000000,0.017799
cmn_Hani,0.002290,0.089297,0.538842,0.044292,0.000101,0.000000,0.000000,0.000000,0.000562,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005360
deu_Latn,0.128698,0.001903,0.000661,0.004671,0.009036,0.000000,0.000514,0.009086,0.006232,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002463
guj_Gujr,0.002107,0.000000,0.007096,0.012418,0.006959,0.000000,0.001256,0.031690,0.007401,0.000000,0.019903,0.001704,0.346849,0.023570,0.000000,0.000000,0.013643
heb_Hebr,0.001054,0.000105,0.002115,0.005851,0.000886,0.005806,0.059821,0.003389,0.096694,0.505050,0.000480,0.006095,0.000000,0.000000,0.000000,0.000000,0.010300


,deu_Latn,jpn_Jpan,cmn_Hani,eng_Latn,tur_Latn,urd_Arab,arb_Arab,hun_Latn,fin_Latn,heb_Hebr,kaz_Cyrl,mon_Cyrl,guj_Gujr,hin_Deva,kor_Hang,pan_Guru,other
1491,0.66,0.0,0.0,0.13,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.09,0.12
1205,0.00,0.0,0.0,0.23,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.55,0.0,0.00,0.22
265,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.91,0.0,0.00,0.09
418,0.00,0.0,0.0,0.12,0.35,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.39,0.00,0.0,0.00,0.14
694,0.00,0.0,0.0,0.08,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.80,0.0,0.00,0.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1046,0.00,0.0,0.0,0.00,0.09,0.0,0.63,0.0,0.0,0.00,0.0,0.06,0.00,0.00,0.0,0.00,0.22
457,0.00,0.0,0.0,0.23,0.00,0.0,0.00,0.0,0.0,0.67,0.0,0.00,0.00,0.00,0.0,0.00,0.10
625,0.89,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.11
425,0.00,0.0,0.0,0.00,0.89,0.0,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.11


In [36]:
languages = ["deu_Latn", "jpn_Jpan", "cmn_Hani", "eng_Latn", "tur_Latn", "urd_Arab", "arb_Arab", "hun_Latn", "fin_Latn", "pan_Guru", "guj_Gujr", "heb_Hebr", "kaz_Cyrl", "mon_Cyrl", "hin_Deva", "kor_Hang", "other"]

In [37]:
langs = []
mse_values_list = []
mae_values_list = []
for lang, mse_values, mae_values in zip(languages, mse, mae):
    # print(f"MSE for {lang}: {mse_values:.5f}")
    langs.append(lang)
    mse_values_list.append(mse_values)
    mae_values_list.append(mae_values)

In [79]:
langs.append("avg")
mse_values_list.append(np.mean(mse))
mae_values_list.append(np.mean(mae))

df_mse = pd.DataFrame.from_dict({
    "lang": langs,
    "mse": mse_values_list,
    "mae": mae_values_list

})
print("avg:", np.mean(mse), " | ", np.mean(mae))

avg: 0.004542619695582728  |  0.02083946632141228
